In [85]:
import pandas as pd
import numpy as np
import os
import sys
import pyodbc
import sqlalchemy

## Loading data
---

In [32]:
# load data from a csv file
df = pd.read_csv('data/Electric_Vehicle_Population_Size_History_By_County.csv')

In [3]:
df.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,April 30 2020,Mason,WA,Truck,0,0,0,20531,20531,0.00
1,April 30 2020,Middlesex,MA,Passenger,1,1,2,101,103,1.94
2,April 30 2020,Monroe,IL,Passenger,1,0,1,4,5,20.00
3,April 30 2020,Moore,NC,Passenger,0,1,1,164,165,0.61
4,April 30 2020,Osceola,FL,Passenger,1,0,1,41,42,2.38


## Connection to the database function
---

In [67]:
# define the connect function
def connect(server, database, username = None, password = None):
    driver = 'SQL Server'
    server = server
    database = database
    if username and password:
        conn_str = f"""
                            DRIVER={driver};
                            SERVER={server};
                            DATABASE={database};
                            UID={username};
                            PWD={password};
                    """
    else:
        conn_str = f"""
                            DRIVER={driver};
                            SERVER={server};
                            DATABASE={database};
                            Trusted_Connection=yes;
                    """
    return pyodbc.connect(conn_str)

## Create a table in the database function
---

In [92]:
# a function to create a table
def create_table(conn, table_name, df):
    # # create a cursor
    # cursor = conn.cursor()
    # # get the column names and types
    # columns = list(df.columns)
    # # get the column types
    # types = list(df.dtypes)
    # # create the table
    # sql_command = f'CREATE TABLE IF NOT EXISTS {table_name} ('
    # # create the sql command for each column
    # for column, type_ in zip(columns, types):
    #     # get the sql command for the column
    #     sql_command += get_db_col(column, type_)
    # # add the closing parenthesis
    # sql_command += ');'
    # # execute the sql command
    # cursor.execute(sql_command)
    # # commit the changes
    # conn.commit()
    # # print a message
    # print(f'Table {table_name} created successfully.')
    # create the table if it does not exist
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f'Table {table_name} created successfully.')
    return

# a function to get the sql command for a column
def get_db_col(column, type_):
    # define the sql command
    db_col = f"{column} {get_sql_type(type_)}"
    return db_col

# a function to get the sql type for a column
def get_sql_type(type_):
    # define the sql type
    sql_type = 'VARCHAR(1000)'
    # check if the type is an int
    if np.issubdtype(type_, np.integer):
        sql_type = 'INT'
    # check if the type is a float
    elif np.issubdtype(type_, np.floating):
        sql_type = 'FLOAT'
    # check if the type is a string
    elif np.issubdtype(type_, np.object_):
        sql_type = 'VARCHAR(1000)'
    elif np.issubdtype(type_, np.datetime64):
        sql_type = 'DATETIME'
    elif np.issubdtype(type_, np.bool_):
        sql_type = 'BIT'
    else:
        print(f'Unknown type: {type_}')
    # return the sql type
    return sql_type

## save json and csv
---

In [65]:
# get column names
columns = df.columns
# strip spaces from column names
columns = [column.strip() for column in columns]
# replace spaces and special characters with underscores
columns = [column.replace(' ', '_') for column in columns]
columns = [column.replace('(', '') for column in columns]
columns = [column.replace(')', '') for column in columns]
columns = [column.replace('/', '_') for column in columns]
columns = [column.replace('-', '_') for column in columns]
columns = [column.replace('%', 'percent') for column in columns]
columns = [column.replace('.', '_') for column in columns]
columns = [column.replace('?', '') for column in columns]
# replace the column names
df.columns = columns
# split data into given percentage of json and sql and csv files given by the user via input
# try and catch block to handle exceptions
try:
    # get percentage of json data from user between 0 and 1
    json_percent = float(input('Enter percentage between 0 and 1 of json data: '))
    # get percentage of sql data from user between 0 and 1
    sql_percent = float(input('Enter percentage between 0 and 1 of sql data: '))
    # get percentage of csv data from user between 0 and 1
    csv_percent = float(input('Enter percentage between 0 and 1 of csv data: '))
    # check if percentages add up to 1
    if json_percent + sql_percent + csv_percent != 1:
        print('Percentages do not add up to 1')
        sys.exit(1)
except ValueError:
    print('Invalid input')
    sys.exit(1)
# split data into json and sql files
# get number of rows
num_rows = df.shape[0]
# get number of json rows
num_json_rows = int(num_rows * json_percent)
# get number of sql rows
num_sql_rows = int(num_rows * sql_percent)
# get number of csv rows
num_csv_rows = num_rows - num_json_rows - num_sql_rows
# get random indices for json rows
json_indices = np.random.choice(num_rows, num_json_rows, replace=False)
# get random indices for sql rows
sql_indices = np.random.choice(num_rows, num_sql_rows, replace=False)
# get random indices for csv rows
csv_indices = np.random.choice(num_rows, num_csv_rows, replace=False)
# get json rows
json_rows = df.iloc[json_indices]
# get sql rows
sql_rows = df.iloc[sql_indices]
# get csv rows
csv_rows = df.iloc[csv_indices]
# write json rows to json file
json_rows.to_json('data/splitted/Electric_Vehicle_Population_Size_History_By_County.json', orient='records', lines=True)
# write csv rows to csv file
csv_rows.to_csv('data/splitted/Electric_Vehicle_Population_Size_History_By_County.csv', index=False, header=False)

## connect to prepare for the sql saving
---

In [90]:
# connect to database
# get server name from user
# server_name = input('Enter server name: ')
driver = 'SQL Server'
server = 'LAPTOP-K8C2EPLP\SQLEXPRESS'
# get database name from user
# db_name = input('Enter database name: ')
database = 'splitted'
# get database user from user
# db_user = input('Enter database user: ')
# get database password from user
# db_password = input('Enter database password: ')
# connect to database
connection_url = f"mssql+pyodbc://{server}/{database}?driver={driver};trusted_connection=yes"

# conn = connect(server, database)
engine = sqlalchemy.create_engine(connection_url)


In [93]:
create_table(engine, 'Test', df)

DBAPIError: (pyodbc.Error) ('IM012', '[IM012] [Microsoft][ODBC Driver Manager] DRIVER keyword syntax error (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [30]:
# detect data types of columns
# Numeric data types: int, float, complex.
# String data types: str.
# Sequence types: list, tuple, range.
# Binary types: bytes, bytearray, memoryview.
# Mapping data type: dict.
# Boolean type: bool.
# Object type: object.
# Set data types: set, frozenset. Python Numeric Data Type.
# get data types of columns
types = df.dtypes
# get column names
columns = df.columns

# remove the dtype(' and ') from the data types
types = types.astype(str).str.replace("dtype('", '').str.replace("')", '')

types.values
# if type is object, convert to the proper type based on the data
for column, type_ in zip(columns, types):
    # check if type is object
    if type_ == 'object':
        # convert to the proper type based on the data
        df[column] = pd.to_datetime(df[column], errors='coerce')
        # check if type is still object
        if df[column].dtype == 'object':
            df[column] = pd.to_numeric(df[column], errors='coerce')
        if df[column].dtype == 'object':
            df[column] = df[column].astype(str)
            
df.dtypes

C:\Users\YouCode\AppData\Local\Temp\ipykernel_15136\3577348123.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors='coerce')
C:\Users\YouCode\AppData\Local\Temp\ipykernel_15136\3577348123.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors='coerce')
C:\Users\YouCode\AppData\Local\Temp\ipykernel_15136\3577348123.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors='coerce')
